# Use case: Sales prediction

*Date: 20/04/2020*  
*Place: Antwerp, Belgium*  
*Data scientist: Anna Sukhareva*  
*Contact: anna@linefeed.be*  

---

## Table of Contents
1 : Business Understanding  
2 : Analytic Approach  
3 : Data Requirements  
4 : Data Collection  
5 : Data Understanding  
6 : Data Preparation  
7 : Modeling  
8 : Evaluation  
9. Model deployment

---

## 1. Business Understanding  
**Problem:**  There is a network comprising 60 retail shops, selling about 20 K unique items. To manage stock and supply chain , we need to suggest an algorithm for sales prediction for each shop, each item_id.  
**Question:** Can we predict sales per item per point of sales?  

## 2. Analytic Approach
Modeling: machine learning algorithms, such as: Linear regression, Random Forest, XGBoost.   
Evaluation: R^2, RMSE.   

## 3. Data reguirements:
Source: Internal data  
Format: csv  
Content: information about sales for each item for each shop, during last 2 years.

## 4. Data collecion

In [ ]:
# Importing libraries

import warnings
warnings.filterwarnings("ignore")

# computation
import numpy as np
import pandas as pd
import itertools
from numpy import inf
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# visualization
%matplotlib inline
import matplotlib.pyplot as plt

# statistic
from pylab import rcParams
import statsmodels.api as sm

# machine learning
from xgboost import XGBRegressor,  plot_importance
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

print('Imported')

In [ ]:
# loading data

items = pd.read_csv(r'.\202004SalesPrediction.data\items.csv',  dtype={
    'item_name': 'str', 
    'item_id': 'int32', 
    'item_category_id': 'int32'
    }
    )
item_categories = pd.read_csv(r'.\202004SalesPrediction.data\item_categories.csv', dtype={
    'item_category_name': 'str', 
    'item_category_id': 'int32'
    }
    )
shops = pd.read_csv(r'.\202004SalesPrediction.data\shops.csv', dtype={
    'shop_name': 'str', 
    'shop_id': 'int32'
    }
    )
test = pd.read_csv(r'.\202004SalesPrediction.data\test.csv', dtype={
    'ID': 'int32', 
    'shop_id': 'int32', 
    'item_id': 'int32'
    }
    )
print('Loaded')

In [ ]:
sales = pd.read_csv(r'.\202004SalesPrediction.data\sales_train.csv', parse_dates=['date'], dtype={
    'date': 'str', 
    'date_block_num': 'int32', 
    'shop_id': 'int32', 
    'item_id': 'int32', 
    'item_price': 'float32', 
    'item_cnt_day': 'int32'
    }
    )
print('Loaded')

In [ ]:
# join datasets

train = sales.join(items, on='item_id', rsuffix='_').join(shops, on='shop_id', rsuffix='_').join(item_categories, on='item_category_id', rsuffix='_').drop(['item_id_', 'shop_id_', 'item_category_id_'], axis=1)

## Stage 5. Data understanding

For efficiency, the analysis of data is placed in a separated file, here - only a head of the dataset.   

In [ ]:
train.head()

**Naming convension:**

Historical features:
- date - date of transaction,  
- date_block_num - month of transaction, enumerated, starting from the first record in 'train' dataset, the month of the 1st transaction has date_num_block == 1,  
- shop_id,  
- item_id,  
- item_price,  
- item_cnt_day - quantity of item in an exact transaction,  

Engineered features:  
- train_monthly - reduced dataset, having records regarding to "shop_id" and "item_id" that appear in deployment,  
- mean_item_price - mean price item during the months (due to discount policy),  
- mean_item_cnt - mean quantity of items sold in 1 month by the shop,  
- transaction - number of deals for the item_id during the month,  
- item_price_unit - unitary item prices as total sales sum // to quantity, to equalize the discount policy,    
- item_cnt_shifted1, item_cnt_shifted2, item_cnt_shifted3 - quantity of sold items in the next/2nd/3rd following month, to reflect the trend,    
- item_trend - difference with 'shifted' column, to reflect the trend,  
- label - historical quantity of items we're going to use to predict during modeling and to evaluate models performance.  

## Stage 6 : Data Preparation  

In [ ]:
# Drop dublicates

#train.duplicated().sum()
train.drop_duplicates(inplace=True)

**Removing outliers  **

In [ ]:
# Column 'item_cnt_day' : removing wholesale deals
train.item_cnt_day.plot.box()
plt.title('Items quantity per transaction: distribution')
plt.show()

I remove all negative (returns) and > 40:

In [ ]:
train = train.loc[train.item_cnt_day > 0]

In [ ]:
train = train.loc[train.item_cnt_day < 40]

In [ ]:
# Column 'item_price' 
train.item_price.plot.box()
plt.title('Items prices: distribution')
plt.show()

Remove negative and below 1 and > 60 000

In [ ]:
train = train.loc[train.item_price >= 1]

In [ ]:
train = train.loc[train.item_price <= 60000]

**Reducing the datset**  

To avoid memory issues on my machine, I reduce train dataset to only "shop_id" and "item_id" that appear in file 'test' dataset:

In [ ]:
test_shop_ids = test['shop_id'].unique()
test_item_ids = test['item_id'].unique()

shorted_train = train[train['shop_id'].isin(test_shop_ids)] 
shorted_train = shorted_train[shorted_train['item_id'].isin(test_item_ids)] 

# saving changes
train_monthly = shorted_train[['date', 'date_block_num', 'shop_id', 'item_category_id', 'item_id', 'item_price', 'item_cnt_day']]

train_monthly.shape

In [ ]:
train_monthly.drop_duplicates(inplace=True)

**Restructuring the dataset**

As we build predictions for month period, we restructure dataset to months time series by groupping data per month and generating monthly statistic:   
- 'mean_item_price - mean price item during the months (due to discount policy),   
- 'mean_item_cnt' - mean quantity of items sold in 1 month by the shop,  
- 'transaction' - number of deals for the item_id during the month  

In [ ]:
# Groupping and renaming columns 

train_monthly = train_monthly.groupby(['date_block_num', 'shop_id', 'item_category_id', 'item_id'], as_index=False).agg({'item_price':['sum', 'mean'], 'item_cnt_day':['sum', 'mean','count']})
train_monthly.columns = ['date_block_num', 'shop_id', 'item_category_id', 'item_id', 'item_price', 'mean_item_price', 'item_cnt', 'mean_item_cnt', 'transactions'] 
train_monthly.head()

In [ ]:
# Extract year and months

train_monthly['year'] = train_monthly['date_block_num'].apply(lambda x: ((x//12) + 2013))
train_monthly['month'] = train_monthly['date_block_num'].apply(lambda x: (x % 12))

train_monthly.head()

**Creating 'label'  **

As we predict sales for the next month, as a label we're going to take a number of sold items of next following month. Sample to explain (add a pic):

In [ ]:
# Creating a label

train_monthly['label'] = train_monthly.sort_values('date_block_num').groupby(['shop_id', 'item_id'])['item_cnt'].shift(-1)

In [ ]:
# Sample - remove after creating a pic
train_monthly.loc[(train_monthly.shop_id == 50) & (train_monthly.item_id == 5822) & (train_monthly.date_block_num == 5)]

** Features engeneering**

We create extra features from data:  
- 'item_price_unit' - unitary item prices as total sales sum // to quntity, to equalize еру discount policy,  
- 'item_cnt_shifted1', 'item_cnt_shifted2', 'item_cnt_shifted3' - quantity of sold items in the next/2nd/3rd folowing month, to reflect the trend,  
- 'item_trend' - difference with 'shifted' column, to reflect the trend

In [ ]:
# item_price_unit

train_monthly['item_price_unit'] = train_monthly['item_price'] // train_monthly['item_cnt']
train_monthly['item_price_unit'].fillna(0, inplace=True)

# remove inf
train_monthly['item_price_unit'].replace(np.inf, 0, inplace=True) 

In [ ]:
# item_cnt_shifted1, item_cnt_shifted2, item_cnt_shifted3

lag_list = [1, 2, 3]

for lag in lag_list:
    ft_name = ('item_cnt_shifted%s' % lag)
    train_monthly[ft_name] = train_monthly.sort_values('date_block_num').groupby(['shop_id', 'item_category_id', 'item_id'])['item_cnt'].shift(lag)
    
    # Fill the empty shifted features with 0
    train_monthly[ft_name].fillna(0, inplace=True)

In [ ]:
# item trend

train_monthly['item_trend'] = train_monthly['item_cnt']

for lag in lag_list:
    ft_name = ('item_cnt_shifted%s' % lag)
    train_monthly['item_trend'] -= train_monthly[ft_name]

train_monthly['item_trend'] /= len(lag_list) + 1

**Resulting dataset:**

In [ ]:
train_monthly.head()

In [ ]:
train_monthly.describe()

## Step 6 : Modeling

### Train - validation - split  

Our train dataset has 33 date_num_block (equal to 33 months), so we divide data as following:    
- train set - date_num_block 3 - 27; we drop first 3 months as they have no data in generated shifted columns,  
- validation set - date_num_block 28-32,   

** Train and validation sets preparation**

In [ ]:
train_set = train_monthly.loc[(train_monthly.date_block_num > 2) & (train_monthly.date_block_num < 28)].copy()
validation_set = train_monthly.loc[(train_monthly.date_block_num >= 28) & (train_monthly.date_block_num != 33)]

We want to generate extra features, as mean for items, shops ans item-shop column. We can't use data from test set, that's why we do it after split:  
For train set:

In [ ]:
def createFeaturesMeansAndAddToTheTable(table):
# Function calculated mean values for shop, item and shop-item in a provided table and writes it down to the table
### Sample to use:  createFeaturesMeansAndAddToTheTable(train_set) ###
    
    df1 = table.groupby(['shop_id']).agg({'label': ['mean']})
    df1.columns = ['shop_mean']
    df1.reset_index(inplace=True)

    # item mean
    df2 = table.groupby(['item_id']).agg({'label': ['mean']})
    df2.columns = ['item_mean']
    df2.reset_index(inplace=True)

    # shop-item mean 
    df3 = table.groupby(['shop_id', 'item_id']).agg({'label': ['mean']})
    df3.columns = ['shop_item_mean']
    df3.reset_index(inplace=True)

    # Add mean features to train set
    table = pd.merge(table, df1, on=['shop_id'], how='left')
    table = pd.merge(table, df2, on=['item_id'], how='left')
    table = pd.merge(table, df3, on=['shop_id', 'item_id'], how='left')

    return table

In [ ]:
train_set = createFeaturesMeansAndAddToTheTable(train_set)

In [ ]:
validation_set = createFeaturesMeansAndAddToTheTable(validation_set)

In [ ]:
# Dropping missed values

train_set.dropna(inplace=True)
validation_set.dropna(inplace=True)

In [ ]:
print(
'Train set:', train_set.shape[0], '\n',
'Validation set:', validation_set.shape[0]
)

Our resulting train and validation datasets look like:

In [ ]:
train_set.head()

In [ ]:
validation_set.head()

Creating train and validation sets and labels

In [ ]:
X_train = train_set.drop(['label', 'date_block_num'], axis=1)
Y_train = train_set['label'].astype(int)

X_validation = validation_set.drop(['label', 'date_block_num'], axis=1)
Y_validation = validation_set['label'].astype(int)

In [ ]:
# Dropping item categories

X_train.drop(['item_category_id'], axis=1, inplace=True)
X_validation.drop(['item_category_id'], axis=1, inplace=True)

### Modeling  
We build 3 models - linear regression, random forest and XGBoost.

### Linear regression

In [ ]:
# Preparing data

# Here, we use some of the features
lr_features = ['item_cnt', 'item_cnt_shifted1', 'item_trend', 'mean_item_cnt', 'shop_mean']

lr_train = X_train[lr_features]
lr_val = X_validation[lr_features]

# Transforming
lr_scaler = MinMaxScaler()
lr_scaler.fit(lr_train)
lr_train = lr_scaler.transform(lr_train)
lr_val = lr_scaler.transform(lr_val)

# Modeling
lr_model = LinearRegression(n_jobs=-1)

# Fitting
lr_model.fit(lr_train, Y_train)

In [ ]:
# Prediction
lr_train_pred = lr_model.predict(lr_train)
lr_val_pred = lr_model.predict(lr_val)

### Random forest

In [ ]:
# Preparing data

# # Here, we use some of the features
rf_features = ['shop_id', 'item_id', 'item_cnt', 'transactions', 'year',
               'item_cnt_shifted1', 'shop_mean', 'item_mean', 'item_trend', 'mean_item_cnt']

rf_train = X_train[rf_features]
rf_val = X_validation[rf_features]

# Modeling
rf_model = RandomForestRegressor(n_estimators=50, max_depth=7, random_state=0, n_jobs=-1)

# Fitting
rf_model.fit(rf_train, Y_train)

In [ ]:
# Prediction
rf_train_pred = rf_model.predict(rf_train)
rf_val_pred = rf_model.predict(rf_val)

### XGBoost

In [ ]:
# Preparing data

# Here, we use some of the features
xgb_features = ['item_cnt', 'item_cnt_shifted1', 'item_cnt_shifted2', 'item_cnt_shifted3', 'shop_mean', 'shop_item_mean', 'item_trend', 'mean_item_cnt']

xgb_train = X_train[xgb_features]
xgb_val = X_validation[xgb_features]

# Modeling
xgb_model = XGBRegressor(max_depth=8, 
                         n_estimators=500, 
                         min_child_weight=1000,  
                         colsample_bytree=0.7, 
                         subsample=0.7, 
                         eta=0.3, 
                         seed=0)
# Fitting
xgb_model.fit(xgb_train, 
              Y_train, 
              eval_metric="rmse", 
              eval_set=[(xgb_train, Y_train), (xgb_val, Y_validation)], 
              verbose=20, 
              early_stopping_rounds=20)

In [ ]:
# Prediction
xgb_train_pred = xgb_model.predict(xgb_train)
xgb_val_pred = xgb_model.predict(xgb_val)

In [ ]:
# Checking features importance
plot_importance(xgb_model)

## Stage 8: Evaluation

In [ ]:
df_RMSE = pd.DataFrame(
    {
        'Model':['Linear Regression', 'Random Forest', 'XGBoost'], 
        'RMSE':[round(np.sqrt(mean_squared_error(Y_validation, lr_val_pred)), 4), round(np.sqrt(mean_squared_error(Y_validation, rf_val_pred)),4), round(np.sqrt(mean_squared_error(Y_validation, xgb_val_pred)),4)],
        'R^2':[round(lr_model.score(lr_val, Y_validation), 2), round(rf_model.score(rf_val, Y_validation), 2), round(xgb_model.score(xgb_val, Y_validation), 2)]
    }
)
df_RMSE = df_RMSE.sort_values('RMSE', ascending=True)
df_RMSE

In [ ]:
# visualization

x = range(df_RMSE.shape[0])
f = plt.figure(figsize=(16,3))

plt.subplot(1,2,1)
R2 = plt.barh(x, df_RMSE['R^2'], color=('darkblue'), zorder=3)
plt.xlabel('Explained data, %')
plt.yticks(x, df_RMSE['Model'])
plt.title('Model performance accuracy R^2, score: the higher - the better')
plt.grid(zorder=0)
plt.xlim(0, 1)

plt.subplot(1,2,2)
RMSE =plt.barh(x, df_RMSE['RMSE'], color=('lightblue'), zorder=3)
plt.xlabel('Error of prediction items quantity, +/- pcs')
plt.yticks(x, df_RMSE['Model'])
plt.title('Model performance error RMSE: the lower- the better')
plt.grid(zorder=0)
plt.show()

Building a table with right labels and corresponding prediction between different models:

**Summary**  
Best performance is shown by random forest model with RMSE +/- 2 pcs.  

## Model deployment
We need to predict sales for some of the shops, for some of the items? for the next month (date_num_bloc = 34). The request to predict look like:

In [ ]:
test.head()

Firstly, ve need to convert data into same structure, as we have in our train and validation sets. 

In [ ]:
# Concating with train and validation sets
train_validation_concated = pd.concat([train_set, validation_set]).drop_duplicates(subset=['shop_id', 'item_id'], keep='last')

In [ ]:
# And merging with our request
#deploy_test
deploy = pd.merge(test, train_validation_concated, on=['shop_id', 'item_id'], how='left', suffixes=['', '_'])

In [ ]:
# adding permanent values
deploy['year'] = 2015
deploy['month'] = 9

# dropping labels
deploy.drop('label', axis=1, inplace=True)

# makin same order of columns like train set
deploy = deploy[X_train.columns]

In [ ]:
# Fill in missing values
sets = [X_train, X_validation, deploy]

In [ ]:
# with the median of each shop   
for dataset in sets:
    for shop_id in dataset['shop_id'].unique():
        for column in dataset.columns:
            shop_median = dataset[(dataset['shop_id'] == shop_id)][column].median()
            dataset.loc[(dataset[column].isnull()) & (dataset['shop_id'] == shop_id), column] = shop_median

In [ ]:
deploy.head()

In [ ]:
# Filling missing values with mean's
deploy.fillna(deploy.mean(), inplace=True)

In [ ]:
# Prepare data
rf_deploy = deploy[rf_features]

# Predict
deploy_prediction = rf_model.predict(rf_deploy)

In [ ]:
# Creating df

deploy_result = test.copy()
deploy_result['date_num_block_34_prediction'] = deploy_prediction.round()
deploy_result.drop('ID', axis=1, inplace=True)
deploy_result

In [ ]:
End of code.